In [1]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [3]:
nx, ny = 101, 101
xmin,xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) * numpy.sin(numpy.pi * X / Lx) * numpy.cos(numpy.pi * Y / Ly))

p0 = numpy.zeros((ny, nx))

p_exact = poisson_solution(x, y, Lx, Ly)

In [4]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return (-4.0 * p[1:-1, 1:-1] + p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1.0
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        Ar[1:-1, 1:-1] = A(r)
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        p = pk + alpha * r
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [6]:
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print('Method of steepest descent: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 9.022504456165771e-17


In [7]:
l2_norm(p, p_exact)

0.9990135263251004

In [8]:
def poisson_2d_steepest_descentA(p0, b, dx, dy,
                                maxiter=20000, rtol=1e-6):
    """
    Solves the 2D Poisson equation on a uniform grid,
    with the same grid spacing in both directions,
    for a given forcing term
    using the method of steepest descent.
    
    The function assumes Dirichlet boundary conditions with value zero.
    The exit criterion of the solver is based on the relative L2-norm
    of the solution difference between two consecutive iterations.

    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats.
    b : numpy.ndarray
        The forcing term as a 2D array of floats.
    dx : float
        Grid spacing in the x direction.
    dy : float
        Grid spacing in the y direction.
    maxiter : integer, optional
        Maximum number of iterations to perform;
        default: 20000.
    rtol : float, optional
        Relative tolerance for convergence;
        default: 1e-6.

    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats.
    ite : integer
        The number of iterations performed.
    conv : list
        The convergence history as a list of floats.
    """
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ar = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        # Compute the residual.
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        # Compute the Laplacian of the residual.
        Ar[1:-1, 1:-1] = A(r)
        # Compute the step size.
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        # Update the solution.
        p = pk + alpha * r
        # Dirichlet boundary conditions are automatically enforced.
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [9]:
p, ites, conv_sd = poisson_2d_steepest_descentA(p0, b, dx, dy,
                                               maxiter=20000,
                                               rtol=1e-10)
print('Method of steepest descent: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


In [10]:
l2_norm(p, p_exact)

8.225076220929745e-05